# Andvaranaut tutorial

## Forward Module (Input distributions specified)

### Latin Hypercube Sampling
Import items form forward module

In [ ]:
from andvaranaut.forward import *
# Magic features for development purposes
%load_ext autoreload
%autoreload 2

User supplies target function, which takes a 1D numpy array of nx inputs and returns a 1D numpy array of ny outputs. They must also supply a list of univariate distributions from scipy stats for each of the nx inputs.

In [ ]:
# Example target function (2 inputs, 2 outputs)
# A more complex target function will produce an input file, 
# execute external code, and perform post-processing on output
from time import sleep
def test_fun(x):
  x1,x2 = x
  sleep(x1)
  return np.array([x1+x2,x1**2*x2])
#   return x1*x2

# Input variable probability distributions
import scipy.stats as st
sample_space = [st.uniform(loc=0,scale=1),\
                st.norm(loc=1,scale=0.5)]

In [ ]:
# Latin hypercube class instance, with correct arguments
l = lhc(nx=2,ny=2,dists=sample_space,target=test_fun)

Sampling makes use of the latin_random function from py-design

In [ ]:
# Sample input distributions by LHC and evaluate target function
samples = 4
l.sample(samples)
print(l.x)
print(l.y)

Parallel execution makes use of the ray package. This also works with SLURM submission if a SLURM script calls a python script with these commands in. (Tutorial will be added at a later date)

In [ ]:
# Can also execute target function evaluation in parallel
l.sample(samples,parallel=True,nproc=4)
print(l.x)
print(l.y)

In [ ]:
# Optionally delete n samples by random indexing
# Can add and delete samples as needed to existing datasets
l.del_samples(samples)
print(l.x)
print(l.y)

Plotting output distributions makes use of the kdeplot function from seaborn

In [ ]:
# Plot output distributions based on kernel density estimation
l.y_dist()